In [37]:
import selenium
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from threading import Thread
from bs4 import BeautifulSoup
import requests
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from datetime import datetime
import re
import time
import sys
from tqdm import tqdm_notebook, tqdm
import selenium.common.exceptions as sce
from selenium.common.exceptions import StaleElementReferenceException

# 사용을 위해 하단으로 내려주시기 바랍니다.
[업데이트 내역]

05/11 : 리뷰 이미지 저장 X, 리뷰데이터 전처리하지 않은 원본 상태로 저장

05/12 : 더보기가 도중에 동작하지 않는 현상 남아있음.

05/15 : 

- 헤더리스 크롤링을 위해 프린트문 추가함
- 서버 에러가 난 업소는 naverplace_review_reject(~).csv 파일로 따로 저장합니다. (재크롤링 필요함)
- 프린트문으로 connection error 문구를 계속 띄운다면 time.sleep이나 스레드 수 조정이 필요합니다.

05/16 :

- 예약자리뷰만 크롤링하도록 코드 수정
- hairdresser와 service 중 하나만 있는 리뷰의 경우 hairdresser와 service에 같은 내용이 들어갑니다
        ex> 'stylist - OO펌' 이 아닌 'stylist' 인 경우 :
        {hairdresser : 'stylist ', service : 'stylist '}
        

In [38]:
#드라이버 실행 => return driver
def startdriver():
    options = webdriver.ChromeOptions()
    prefs = {
        'profile.default_content_setting_values': {
              'cookies': 2, 'images': 2, 'plugins': 2
            , 'popups': 2, 'geolocation': 2, 'notifications': 2
            , 'auto_select_certificate': 2, 'fullscreen': 2, 'mouselock': 2
            , 'mixed_script': 2, 'media_stream': 2, 'media_stream_mic': 2
            , 'media_stream_camera': 2, 'protocol_handlers': 2, 'ppapi_broker': 2
            , 'automatic_downloads': 2, 'midi_sysex': 2, 'push_messaging': 2
            , 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2, 'protected_media_identifier': 2
            , 'app_banner': 2, 'site_engagement': 2, 'durable_storage': 2
        }
    }

    options.add_experimental_option('prefs', prefs)
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36')
    options.add_argument("Accept=text/html,application/xhtml+xml,application/xml;q=0.9,imgwebp,*/*;q=0.8")
    options.add_argument("start-maximized")
    options.add_argument('--no-sandbox')
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument('headless')
    options.add_argument("disable-gpu")
    options.add_argument("--disable-web-security")
    driver = webdriver.Chrome(driverpath, options=options)
    return driver

In [39]:
def get_driver(driver, index):
    driver.get('https://m.place.naver.com/hairshop/'+str(salonDF.store_id_only_num[index])+'/review/booking')
    status = check_status(index)
    return driver, status

#더보기 버튼 클릭하는 함수
def click_more_btn(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    btn = driver.find_element(By.XPATH, '//div[@class="_2kAri "]//a[@role="button"]')
    EC.element_to_be_clickable((By.XPATH, '//div[@class="_2kAri "]//a[@role="button"]'))
    btn.click()


def check_btn(driver):
    try:
        # 변수 초기화
        pre_count = 0
        count_dup = 0
        reviews = []
        
        while 1:
            #처음 클릭
            click_more_btn(driver)
            time.sleep(1*np.random.rand())
            #화면에 생성된 리뷰
            reviews = driver.find_elements(By.XPATH, '//div[@class="_2eFND"]')

            if pre_count == len(reviews):
                count_dup += 1
            else:
                count_dup = 0

            if count_dup < 10:
                pre_count = len(reviews)
            else:
                print('같은 페이지 10번 반복, 스크롤을 마칩니다.')
                break
            
    except StaleElementReferenceException :
        print('StaleElementReferenceException')
        
    except sce.NoSuchElementException :
        print('NoSuchElementException')

    finally :
        reviews = driver.find_elements(By.XPATH, '//div[@class="_2eFND"]')
        print('예약자 리뷰 개수 :', len(reviews)," 스크롤 종료")
        return driver
def find_elem(driver):
    return driver.find_elements(By.XPATH, '//div[@class="_2eFND"]')

#driver => dict list->list
def crawling_rev(driver_, index:int) :
    driver, status = get_driver(driver_, index)
    driver = check_btn(driver)
    dictlist=return_dictlist(driver, index)
    return dictlist, driver, status

#driver-find_elem => return reviewlist
def return_dictlist(driver, index) :
    reviews= find_elem(driver)
    reviewlist=[]
    #salon_id
    store_id_only_num = salonDF['store_id_only_num'][index]
    #salon_name
    store_name = salonDF['store_name'][index]
    for rev in tqdm(reviews) :

        #별점 가져오기
        score = rev.find_element(By.CLASS_NAME, '_35U3m').text
        
        #미용사 가져오기
        try :
            hairdresser = rev.find_element(By.CLASS_NAME, '_2huo5').text.split('-')[0]
        except sce.NoSuchElementException:
            hairdresser = np.nan
        
        #서비스 가져오기
        try :
            service = rev.find_element(By.CLASS_NAME, '_2huo5').text.split('-')[-1]
        except sce.NoSuchElementException:
            service = np.nan    
        
        #리뷰 가져오기
        try:
            review = rev.find_element(By.CLASS_NAME, 'WoYOw').text 
        except sce.WebDriverException as e:
            review = webDriverExceptionError(e)
        finally : 
            review = deleteEmoji(review)
        
        #작성자 가져오기
        reviewer_date = rev.find_element(By.CLASS_NAME, '_31KaU').find_elements(By.TAG_NAME, 'span')
        reviewer_id = reviewer_date[0].text
        
        #방문일자 가져오기
        visit_date = reviewer_date[1].text
        
        #딕셔너리로 저장하기
        review_info = {'store_id_only_num':store_id_only_num, 'store_name':store_name
                      , 'score':score, 'hairdresser':hairdresser, 'service':service
                      , 'review':review, 'reviewer_id':reviewer_id
                      , 'visit_date':visit_date}
        reviewlist.append(review_info)
        print(f'예약자 리뷰 : {review_info}')
    
    return reviewlist

#리뷰 이모지 제거
def deleteEmoji(text:str):
    p=re.compile(r"[\\u\w]{1,}")
    compiled_text = p.findall(text)
    final_text=' '.join(compiled_text)
    return final_text

#threading 
def threading_craw(start, end, n_thread) :
    step = np.linspace(start, end, n_thread+1, dtype=int)
    ths = []
    
    for i in range(0,n_thread) :
        th = Thread(target=do_crw, args=(step[i], step[i+1]))
        ths.append(th)
        time.sleep(np.random.rand())
        th.start()
    
    for th in ths :
        th.join()
        
#한번에 여러 업소 크롤링
def do_crw(start, end) :
    driver_ = startdriver()
    rejectDF = pd.DataFrame(columns=salonDF.columns)
    dflist=[]

    for i in tqdm_notebook(range(start, end)):
        df, driver, status = crawling_rev(driver_, i)
        dflist+=df
        DataFrame(dflist).to_csv("./naverplace_booking_review(%i-%i).csv" %(start, end-1), sep=',', encoding='utf-8-sig')
        print(f'인덱스 번호 : {i} 예약자 리뷰 크롤링 완료')
        
        if status == 0 :
            rejectDF = rejectDF.append(salonDF[i:i+1])
            DataFrame(rejectDF).to_csv("./naverplace_review_reject(%i-%i).csv" %(start, end-1), sep=',', encoding='utf-8-sig')
       
        time.sleep(10)
    driver.quit()

    
#webDriverExceptionError
def webDriverExceptionError(error) : 
    unitext = error.msg.split('value":"')[-1].split('(Session')[0]
    encoded = unitext.encode('utf-8')
    text = encoded.decode('unicode_escape')
    splited = text.split(' ')
    emoji_pattern = re.compile(
            u"(\ud83d)|"
            u"(\ud83c)|"
            u"(\ud83d[\ude00-\ude4f])|"  # emoticons
            u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
            u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
            u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
            u"(\ud83c[\udde0-\uddff])"   # flags (iOS)
            "+", flags=re.UNICODE)
    rejoined = ' '.join(list(map(lambda x: x if not emoji_pattern.search(x) else '', splited)))
    final_text = rejoined.strip()
    return final_text


#접속 상태 확인
def check_status(index):
    response = requests.get('https://m.place.naver.com/hairshop/'+str(salonDF.store_id_only_num[index])+'/review')
    status = 0
    
    if response.status_code != 200 :
        print("인덱스 번호 : ",index, "connection error")
        status = 0
        time.sleep(10)
        return status
    else :
        print("인덱스 번호 : ",index, '접속 성공!')
        status = 1
        return status

### 아래 셀만 수정하시면 됩니다.
### 수정해야할 것 : 
    - path : naver_place_count_total_2020-05-01_final.csv 파일 위치를 적어주시면 됩니다.
    - driverpath : chromedriver.exe 파일의 위치를 적어주시면 됩니다.
    - start : 시작번호 (1입력할 경우 1번부터 저장됩니다.)
    - end : 끝번호(100입력할 경우 99번까지 저장됩니다.)
    - n_thread : 생성할 스레드 수 (한번에 띄울 창 개수, 자신의 컴퓨터 사양에 맞게 조정하십시오 ; 3개 추천)
- 현택호 0\~10 :14292 (index : 0~9)
- 한주리 10\~40 : 16175 (index : 10~39)
- 이한울 40\~140 : 25250 (index : 40~139)
- 성민호 140\~380 : 30542 (index : 140~379)
- 서동원 380\~2727 : 41041 (index : 380~2726)    

In [40]:
#저장 경로 지정하기
path = './naver_place_count_total_2020-05-01_final.csv'
driverpath = 'D:/네이버 플레이스 리뷰 크롤링/BookingReviewCrw/81win32/chromedriver.exe'
salonDF = pd.read_csv(path, index_col=0)

In [41]:
#시작번호, 끝번호, 생성할 스레드 수(창)
n_start, n_end, n_thread= 140, 380, 3
threading_craw(n_start, n_end, n_thread)

Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\1\anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\1\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-39-ab9ae40aa5e4>", line 132, in do_crw
    driver_ = startdriver()
  File "<ipython-input-38-402b3e69f9c0>", line 28, in startdriver
    driver = webdriver.Chrome(driverpath, options=options)
  File "C:\Users\1\anaconda3\lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 81, in __init__
    desired_capabilities=desired_capabilities)
  File "C:\Users\1\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 157, in __init__
    self.start_session(capabilities, browser_profile)
  File "C:\Users\1\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 252, in start_session
    response = self.execute(Command.NEW_SESSION, parameters)
  File "C:\Users\1\a

## 저장 후 확인 작업들

In [8]:
df = pd.read_csv('./place_review/2020-05-16_place_blog-error (0-10000).csv')
df.head()
# aa = set(df['store_id_only_num'].values)
# salonDF[salonDF['store_id_only_num'].isin(aa)]
# salonDF[n_start:n_end]

,Unnamed: 0,store_id,store_name,link,original_index
0,0,37818441,에이치스타일 가로수길,http://post.naver.com/viewer/postView.nhn?volu...,1295


In [14]:
from glob import glob

In [17]:
reject_review=glob('./*naverplace_review_reject*')
reject_review

['.\\naverplace_review_reject(140-187).csv',
 '.\\naverplace_review_reject(188-235).csv',
 '.\\naverplace_review_reject(236-283).csv',
 '.\\naverplace_review_reject(284-331).csv',
 '.\\naverplace_review_reject(332-379).csv']

In [20]:
df01,df02,df03,df04,df05 = pd.read_csv('./naverplace_review_reject(140-187).csv')\
,pd.read_csv('./naverplace_review_reject(188-235).csv'),\
pd.read_csv('./naverplace_review_reject(236-283).csv'),\
pd.read_csv('./naverplace_review_reject(284-331).csv'),\
pd.read_csv('./naverplace_review_reject(332-379).csv')


In [25]:
len(df01)

44

In [26]:
reject_review_final = pd.concat([df01,df02,df03,df04,df05])
reject_review_final

,Unnamed: 0,store_id,store_id_only_num,store_name,booking,receipt,blog,total
0,144,s36878425,36878425,보그헤어 신도림역점,179.0,1.0,10.0,190.0
1,145,s1445855462,1445855462,아버헤어 홍대 롯데호텔점,175.0,3.0,10.0,188.0
2,146,s34052371,34052371,H STYLE 헤어살롱 노량진점,176.0,1.0,10.0,187.0
3,147,s1311049269,1311049269,보그헤어 창동역점,179.0,1.0,7.0,187.0
4,148,s37207595,37207595,아이디헤어 은평뉴타운점,172.0,3.0,12.0,187.0
...,...,...,...,...,...,...,...,...
36,375,s18714950,18714950,살롱에이 중대점,93.0,1.0,10.0,104.0
37,376,s1258477274,1258477274,로이드밤 구로점,97.0,1.0,6.0,104.0
38,377,s66838411,66838411,리안헤어 신길역점,97.0,1.0,6.0,104.0
39,378,s1853807979,1853807979,하움,93.0,1.0,10.0,104.0


In [31]:
bb = glob('./*naverplace_booking_review*')


['.\\naverplace_booking_review(140-187).csv',
 '.\\naverplace_booking_review(140~380).csv',
 '.\\naverplace_booking_review(188-235).csv',
 '.\\naverplace_booking_review(236-283).csv',
 '.\\naverplace_booking_review(284-331).csv',
 '.\\naverplace_booking_review(332-379).csv']

In [ ]:
paths_naverplace_booking_review = glob('./*naverplace_booking_review*')
booking_review = pd.concat([pd.read_csv(path, index_col=0) for path in paths_naverplace_booking_review])

In [36]:
booking_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54600 entries, 0 to 6337
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   store_id_only_num  54600 non-null  int64  
 1   store_name         29004 non-null  object 
 2   score              54600 non-null  float64
 3   hairdresser        54580 non-null  object 
 4   service            54600 non-null  object 
 5   review             54568 non-null  object 
 6   reviewer_id        54600 non-null  object 
 7   visit_date         54600 non-null  object 
 8   Unnamed: 0.1       25596 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 4.2+ MB


In [35]:
booking_review.groupby('store_name').count()

,store_id_only_num,score,hairdresser,service,review,reviewer_id,visit_date,Unnamed: 0.1
store_name,,,,,,,,
ZS헤어살롱,1065,1065,1065,1065,1065,1065,1065,0
그레,880,880,880,880,880,880,880,0
더감동 미아본점,950,950,950,950,949,950,950,0
로이드밤 구파발역점,230,230,230,230,230,230,230,0
리안헤어 문정법조타운점,1735,1735,1735,1735,1735,1735,1735,0
리안헤어 방이역점,1201,1201,1201,1201,1201,1201,1201,0
박승철헤어스투디오 독산롯데캐슬점,983,983,983,983,983,983,983,0
박준뷰티랩 뚝섬유원지역점,280,280,280,280,280,280,280,0
박준뷰티랩 송파헬리오시티점,1351,1351,1351,1351,1350,1351,1351,0


In [27]:
reject_review_final.to_csv('./reject_review_final')

In [30]:
pd.read_csv('./reject_review_final')

,Unnamed: 0,Unnamed: 0.1,store_id,store_id_only_num,store_name,booking,receipt,blog,total
0,0,144,s36878425,36878425,보그헤어 신도림역점,179.0,1.0,10.0,190.0
1,1,145,s1445855462,1445855462,아버헤어 홍대 롯데호텔점,175.0,3.0,10.0,188.0
2,2,146,s34052371,34052371,H STYLE 헤어살롱 노량진점,176.0,1.0,10.0,187.0
3,3,147,s1311049269,1311049269,보그헤어 창동역점,179.0,1.0,7.0,187.0
4,4,148,s37207595,37207595,아이디헤어 은평뉴타운점,172.0,3.0,12.0,187.0
...,...,...,...,...,...,...,...,...,...
205,36,375,s18714950,18714950,살롱에이 중대점,93.0,1.0,10.0,104.0
206,37,376,s1258477274,1258477274,로이드밤 구로점,97.0,1.0,6.0,104.0
207,38,377,s66838411,66838411,리안헤어 신길역점,97.0,1.0,6.0,104.0
208,39,378,s1853807979,1853807979,하움,93.0,1.0,10.0,104.0


In [9]:
df['link']

0    http://post.naver.com/viewer/postView.nhn?volu...
Name: link, dtype: object